In [4]:
# Pre-flight checks (quotas, permissions, users, etc.)
# https://docs.openshift.com/rosa/rosa_cli/rosa-get-started-cli.html#rosa-init_rosa-getting-started-cli
!rosa init -y

I: Admin user 'osdCcsAdmin' created successfully!
I: Validating SCP policies for 'osdCcsAdmin'...
I: AWS SCP policies ok
I: Validating cluster creation...
I: Cluster creation valid
I: Verifying whether OpenShift command-line tool is available...
I: Current OpenShift Client Version: 4.8.9


In [5]:
# Use a unique id to identify resources)
import os
unique_id = os.getenv("UNIQUE_ID","rhnb")
unique_id

'rhnb'

In [6]:
# Define cluster name
cluster_name= unique_id+"-rosa"
cluster_name

'rhnb-rosa'

In [1]:
# Before provisioning a cluster, make sure to understand pricing and security implications.
# https://aws.amazon.com/rosa/pricing/

In [8]:
# Create a cluster with default settings
! rosa create cluster --cluster-name={cluster_name}

I: Creating cluster 'rhnb-rosa'
I: To view a list of clusters and their status, run 'rosa list clusters'
I: Cluster 'rhnb-rosa' has been created.
I: Once the cluster is installed you will need to add an Identity Provider before you can login into the cluster. See 'rosa create idp --help' for more information.
I: To determine when your cluster is Ready, run 'rosa describe cluster -c rhnb-rosa'.
I: To watch your cluster installation logs, run 'rosa logs install -c rhnb-rosa --watch'.
Name:                       rhnb-rosa
ID:                         1pmvjekmufhh8855dkbjc94arfn1222q
External ID:                
OpenShift Version:          
Channel Group:              stable
DNS:                        rhnb-rosa.b1w6.p1.openshiftapps.com
AWS Account:                269733383066
API URL:                    
Console URL:                
Region:                     ap-northeast-1
Multi-AZ:                   false
Nodes:
 - Control plane:           3
 - Infra:                   2
 - Compute:   

In [9]:
! rosa list clusters --output json | jq -r '.[0]'

{
  "kind": "Cluster",
  "id": "1pmvjekmufhh8855dkbjc94arfn1222q",
  "href": "/api/clusters_mgmt/v1/clusters/1pmvjekmufhh8855dkbjc94arfn1222q",
  "api": {
    "listening": "external"
  },
  "aws": {
    "private_link": false
  },
  "aws_infrastructure_access_role_grants": {
    "items": []
  },
  "ccs": {
    "kind": "CCS",
    "disable_scp_checks": false,
    "enabled": true
  },
  "dns": {
    "base_domain": "b1w6.p1.openshiftapps.com"
  },
  "addons": {
    "items": []
  },
  "billing_model": "standard",
  "cloud_provider": {
    "kind": "CloudProviderLink",
    "id": "aws",
    "href": "/api/clusters_mgmt/v1/cloud_providers/aws"
  },
  "creation_timestamp": "2022-01-13T15:39:43Z",
  "disable_user_workload_monitoring": false,
  "display_name": "rhnb-rosa",
  "etcd_encryption": true,
  "external_id": "aff16b18-7665-460b-b65a-7b87c438dae8",
  "external_configuration": {
    "labels": {
      "items": []
    },
    "syncsets": {
      "items": []
    }
  },
  "flavour": {
    "kind": "

In [11]:
# Wait until cluster is ready
# TODO: Use waiter from https://github.com/openshift/rosa/pull/503
# rosa wait-for -c id42-cluster
states=! rosa list clusters --output json | jq -r ".[].status.state"
states=states.s
states

'ready'

Installing a cluster with default settings takes around half an hour.

Good time to watch a video on the [OpenShift Youtube channel](https://www.youtube.com/c/OpenShift) :)

In [12]:
# Create admin user
# TODO: Use password option from PR (-p Masterkey123)
! rosa create admin --cluster="{cluster_name}" | tee create_admin.log

W: It is recommended to add an identity provider to login to this cluster. See 'rosa create idp --help' for more information.
I: Admin account has been added to cluster 'rhnb-rosa'.
I: Please securely store this generated password. If you lose this password you can delete and recreate the cluster admin user.
I: To login, run the following command:

   oc login https://api.rhnb-rosa.b1w6.p1.openshiftapps.com:6443 --username cluster-admin --password B46vg-UEfUS-hmQjc-8SFbs

I: It may take up to a minute for the account to become active.


In [13]:
# Check login command
# TODO: Use choice of password
!cat create_admin.log 

I: Admin account has been added to cluster 'rhnb-rosa'.
I: Please securely store this generated password. If you lose this password you can delete and recreate the cluster admin user.
I: To login, run the following command:

   oc login https://api.rhnb-rosa.b1w6.p1.openshiftapps.com:6443 --username cluster-admin --password B46vg-UEfUS-hmQjc-8SFbs

I: It may take up to a minute for the account to become active.


In [ ]:
# Check login command
# TODO: Use choice of password
!cat create_admin.log 

In [ ]:
# Wait for admin user to be active, usually takes a couple minutes
# TODO: Add a waiter command for that

In [16]:
# Login (--insecure-skip-tls-verify=true)
! oc login https://api.rhnb-rosa.b1w6.p1.openshiftapps.com:6443 --username cluster-admin --password B46vg-UEfUS-hmQjc-8SFbs --insecure-skip-tls-verify=true

Login successful.

You have access to 90 projects, the list has been suppressed. You can list all projects with 'oc projects'

Using project "default".


In [17]:
# Verify cluster authentication
! oc whoami

cluster-admin


In [18]:
# Ping for version
!oc version

Client Version: 4.8.9
Server Version: 4.9.12
Kubernetes Version: v1.22.3+e790d7f


In [19]:
# Check cluster status
!oc status

In project default on server https://api.rhnb-rosa.b1w6.p1.openshiftapps.com:6443

svc/openshift - kubernetes.default.svc.cluster.local
svc/kubernetes - 172.30.0.1:443 -> 6443

View details with 'oc describe <resource>/<name>' or list resources with 'oc get all'.


In [20]:
# Check running resources
!oc get nodes

NAME                                              STATUS                        ROLES          AGE   VERSION
ip-10-0-135-60.ap-northeast-1.compute.internal    NotReady,SchedulingDisabled   master         36m   v1.22.3+e790d7f
ip-10-0-142-182.ap-northeast-1.compute.internal   Ready                         master         36m   v1.22.3+e790d7f
ip-10-0-164-25.ap-northeast-1.compute.internal    Ready                         infra,worker   11m   v1.22.3+e790d7f
ip-10-0-176-56.ap-northeast-1.compute.internal    Ready                         worker         27m   v1.22.3+e790d7f
ip-10-0-179-154.ap-northeast-1.compute.internal   Ready                         worker         27m   v1.22.3+e790d7f
ip-10-0-183-184.ap-northeast-1.compute.internal   Ready                         master         36m   v1.22.3+e790d7f
ip-10-0-185-16.ap-northeast-1.compute.internal    Ready                         infra,worker   11m   v1.22.3+e790d7f


In [ ]:
# Decomissioning

In [ ]:
# delete cluster
!rosa delete cluster -y -c "{cluster_name}"

In [ ]:
#  delete IAM admin user
!rosa init -y --delete